Устанавливаем все необходимые библиотеки.

In [1]:
from google.colab import files
from IPython.display import clear_output

In [2]:
!pip install pycodestyle pycodestyle_magic
!pip install flake8
clear_output()

In [3]:
%load_ext pycodestyle_magic

In [4]:
!pip install -U -q PyDrive

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [6]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Загружаем данные.

In [ ]:
whole_table_with_lemm = drive.CreateFile(
    {'id': '1rKrNCmgWPReJujCoZCDlhAb66yiUZ_by'})
whole_table_with_lemm.GetContentFile('whole_table_with_lemm.csv')

Импортируем все необходимые библиотеки.

In [22]:
import re
import nltk
import joblib

import pandas as pd
import numpy as np

from pprint import pprint
from ast import literal_eval
from datetime import datetime
from string import punctuation

from tqdm.notebook import tqdm

from nltk.corpus import stopwords

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

clear_output()

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
stop_words = set(stopwords.words('russian'))

# Цель исследования
Тематическое моделирование корпуса текстов: связь тем дневниковых записей с периодом времени написания.
# Данные
**Meet your dataset: дневниковые записи** [(источник)](https://t.me/c/1358367494/782)<br>
<br>
**Откуда у нас дневники?**<br>
Наши данные - это причёсанный дамп [сайта "Прожито"](https://prozhito.org) от апреля 2019 года. Таблицы содержит несколько сотен тысяч записей за большой отрезок времени (от XVIII до XXI века, преимущественно — XX век), так что вам будет, где развернуться ;).<br>
<br>
**Как устроен датасет?**<br>
У нас есть две таблицы, `whole_table.csv` и `whole_table_with_lemm.csv`. Вторая отличается наличием колонки c лемматизированными (т.е. с приведенными в начальную форму словами) mystem записями.<br>
В обеих таблицах есть колонка:
*   `notes` - содержит сами дневниковые записи
*   `dates` - дата записи в формате год/месяц/день
*   `id`  -  айдишник автора (не записи!)
*   `author` - имя автора записи
<br>

Практически все поля заполнены, у некоторых отсутствует дата (так как таблица отсортирована по датам, они в самом начале).<br>
*От авторов исследования: даты отформатированы как YYYY/(M)M/(D)D, отсутствовать могут день, месяц или год, отсутствующие элементы даты заменены нулями.*
# Препроцессинг

In [ ]:
# напишем отдельную функцию парсинга дат для записей раньше 20 века
# или записей у которых отсутствует год, месяц или число
# исходя из целей исследования, такие записи неинформативны и их нужно удалять

def date_parser(date):
    if date and re.match(r'^19\d{2}/[1-9]\d?/[1-9]\d?$', date):
        date_split = [int(elem) for elem in date.split('/')]
        if date_split[0] >= 1905:
            return datetime(year=date_split[0],
                            month=date_split[1],
                            day=date_split[2])
    return np.nan

In [ ]:
df = pd.read_csv('whole_table_with_lemm.csv', sep='\t', index_col=0,
                 usecols=[0, 2, 3], parse_dates=['dates'],
                 date_parser=date_parser).rename(
                     columns={'notes\n': 'lemm', 'dates': 'date'})

In [ ]:
df.head()

,lemm,date
0,воскресение. \nвчера приезжать слава навеща...,NaT
1,"ночь мы достигать банка антон-дорн, в датский ...",NaT
2,суббота. москва. ехать в селище.\n,NaT
3,18 август его королевский величество отправлят...,NaT
4,"на следующий день, 19 август, прибывать в остр...",NaT


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384526 entries, 0 to 384525
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   lemm    384525 non-null  object        
 1   date    302547 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 8.8+ MB


In [ ]:
!rm whole_table_with_lemm.csv

Удаляем ненужные записи, делаем столбец `date` индексом.

In [ ]:
df = df.dropna().set_index('date')

In [ ]:
df.head()

,lemm
date,
1905-01-01,"в 7 час утро — 4°, туман с дым, тихо. сегодня ..."
1905-01-01,с ранний утро японец опять открывать огонь раз...
1905-01-01,день новый год проходить благополучно. японец ...
1905-01-01,"вчера говорить, что поражать перемена, который..."
1905-01-01,воскресение. японский выя год \nобычный вос...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 302546 entries, 1905-01-01 to 1999-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lemm    302546 non-null  object
dtypes: object(1)
memory usage: 4.6+ MB


## Токенизация

In [ ]:
punctuation + '— \n'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~— \n'

In [ ]:
regex = r'[!\"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~— \n]+'

In [ ]:
def tokenization(text):
    return [elem for elem in re.split(regex, text)
            if elem and elem not in stop_words]

In [ ]:
df['lemm'] = df['lemm'].apply(tokenization)
clear_output()

In [ ]:
df.head()

,lemm
date,
1905-01-01,"[7, час, утро, 4°, туман, дым, тихо, сегодня, ..."
1905-01-01,"[ранний, утро, японец, открывать, огонь, разом..."
1905-01-01,"[день, новый, год, проходить, благополучно, яп..."
1905-01-01,"[вчера, говорить, поражать, перемена, который,..."
1905-01-01,"[воскресение, японский, выя, год, обычный, вос..."


Сохраняем и загружаем на гуглдиск готовые к topic modelling данные.

In [ ]:
df.to_csv('prozhito_lemmas.csv')

In [ ]:
!zip prozhito_lemmas.zip prozhito_lemmas.csv

  adding: prozhito_lemmas.csv (deflated 78%)


In [ ]:
!rm prozhito_lemmas.csv

In [ ]:
files.download('prozhito_lemmas.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки готовой к topic modelling таблицы

# prozhito_lemmas = drive.CreateFile(
#    {'id': '1_t6vdR2iREGMdNNtuYMc09anV0YpbaSs'})
# prozhito_lemmas.GetContentFile('prozhito_lemmas.zip')

Функция для загруженного из гуглдиска датафрейма.

In [23]:
def data_to_list(filename):
    df = pd.read_csv(filename)
    df['lemm'] = df['lemm'].apply(literal_eval)
    clear_output()
    return df

## Периодизация
Поскольку основная часть текстов корпуса приходится на советскую эпоху, решено было выделить следующие периоды:
*   *янв. 1905 – янв. 1907* (первая революция)
*   *февр. 1907 – май 1914* (период между революциями)
*   *июнь 1914 – октябрь 1918* (Первая мировая война и две главные революции в России)
*   *ноябрь 1918 – декабрь 1927* (гражданская война, продразверстка, НЭП)
*   *январь 1928 – август 1939* (коллективизация, Большой террор)
*   *сентябрь 1939 – май 1945* (Вторая мировая война)
*   *1946 – 1963* (промежуточный период истории, заканчивается «оттепелью»)
*   *1964 – май 1987* (застой)
*   *июнь 1987 – июнь 1990* (перестройка и развал СССР)
*   *1991 – 2001* (после СССР, «лихие 90-е»)

In [ ]:
df = [
    df.loc[
        pd.Timestamp('1905-1-1 00:00:00'): pd.Timestamp('1907-1-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1907-2-1 00:00:00'): pd.Timestamp('1914-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1914-6-1 00:00:00'): pd.Timestamp('1918-10-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1918-11-1 00:00:00'): pd.Timestamp('1927-12-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1928-1-1 00:00:00'): pd.Timestamp('1939-8-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1939-9-1 00:00:00'): pd.Timestamp('1945-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1946-1-1 00:00:00'): pd.Timestamp('1963-12-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1964-1-1 00:00:00'): pd.Timestamp('1987-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1987-6-1 00:00:00'): pd.Timestamp('1990-6-30 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1991-1-1 00:00:00'): pd.Timestamp('2001-12-31 23:59:59')]
       ]

In [25]:
df[1].head()

,date,lemm
0,1907-02-01,"[пятница, чудный, погода, 1, солнце, ветер, па..."
1,1907-02-01,"[вчера, ната, звонить, шлиппе, сказать, сергей..."
2,1907-02-01,"[пятница, приходиться, позвать, шенгелидзе, т,..."
3,1907-02-01,"[четверг, писать, 2, февраль, доклад, военный,..."
4,1907-02-01,"[тетя, васино, рождение, тетя, мила, ходить, к..."


In [26]:
df[1]['lemm'][0]

['пятница',
 'чудный',
 'погода',
 '1',
 'солнце',
 'ветер',
 'пахнуть',
 'весна',
 'это',
 'удивительно',
 'приятно',
 'завтрак',
 'лена',
 'ходить',
 'гулять',
 'обед',
 'время',
 'проходить',
 'совершенно',
 'бесцельно',
 'лень',
 'приниматься',
 'бродить',
 'угол',
 'угол',
 'таланцев',
 'пробирать',
 'немножко',
 'митя',
 'лень',
 'решать',
 'вообще',
 'обращаться',
 'строгий',
 'позволять',
 'александра',
 'арсеньевна',
 'рассказывать',
 'нижний',
 'думать',
 'открывать',
 'женский',
 'педагогический',
 'курсы',
 'хороший',
 'непременно',
 'поступать',
 'хотеться',
 'учиться',
 'вечер',
 'лихорадка',
 '37',
 '5',
 'таки',
 'пойти',
 'театр',
 'лена',
 'вася',
 'концерт',
 'польза',
 'общежитие',
 'гимназия',
 'концерт',
 'производить',
 'ровно',
 'никакой',
 'впечатление',
 'скучно',
 'антракт',
 'видеть',
 'эрю',
 'зина',
 'марья',
 'миха',
 'многий',
 'знакомый']

In [15]:
period = ['first revolution', 'between revolutions', 'WWI', 'civil war',
          'great purge', 'WWII', 'before ottepel', 'stagnation','perestroyka',
          '90s']

In [27]:
for i, time in enumerate(period):
    print(f'{time}:{" " * (20 - len(time))}\t{df[i].shape[0]} entries')

first revolution:    	5821 entries
between revolutions: 	18761 entries
WWI:                 	20362 entries
civil war:           	35981 entries
great purge:         	37175 entries
WWII:                	62686 entries
before ottepel:      	39742 entries
stagnation:          	58121 entries
perestroyka:         	6723 entries
90s:                 	13283 entries


Сохраняем и загружаем на гуглдиск разбитые на периоды данные.

In [ ]:
for i, table in tqdm(enumerate(df)):
    table.to_csv(f'{i + 1}_{period[i].replace(" ", "_")}.csv')

In [ ]:
!zip prozhito_periodization.zip *.csv

  adding: 10_90s.csv (deflated 78%)
  adding: 1_first_revolution.csv (deflated 78%)
  adding: 2_between_revolutions.csv (deflated 78%)
  adding: 3_WWI.csv (deflated 78%)
  adding: 4_civil_war.csv (deflated 77%)
  adding: 5_great_purge.csv (deflated 78%)
  adding: 6_WWII.csv (deflated 78%)
  adding: 7_before_ottepel.csv (deflated 78%)
  adding: 8_stagnation.csv (deflated 78%)
  adding: 9_perestroyka.csv (deflated 78%)


In [ ]:
files.download('prozhito_periodization.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# код для загрузки архива таблиц для каждого периода

# prozhito_periodization = drive.CreateFile(
#    {'id': '1LcrY9pNxXHa9xLJ4jGnm8lXVbegFodUQ'})
# prozhito_periodization.GetContentFile('prozhito_periodization.zip')

# LDA with Gensim

In [ ]:
models = []

In [39]:
for table in df[8:]:
    data = list(table['lemm'])
    id2word = Dictionary(data)
    corpus = [id2word.doc2bow(text) for text in data]
    model = LdaModel(corpus, num_topics=10, id2word=id2word, passes=15,
                     iterations=100, per_word_topics=True)
    models.append(model)
    clear_output()
    print(f'Моделей обучено: {len(models)}')

Моделей обучено: 10


Сохраняем и загружаем на гуглдиск обученные модели с наборами тем для каждого периода.

In [42]:
joblib.dump(models, 'prozhito_topic_modelling.pkl')

['prozhito_topic_modelling.pkl']

In [43]:
!zip prozhito_topic_modelling.zip prozhito_topic_modelling.pkl

  adding: prozhito_topic_modelling.pkl (deflated 54%)


In [44]:
files.download('prozhito_topic_modelling.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки обученных моделей

# prozhito_topic_modelling = drive.CreateFile(
#    {'id': '1og0bwBkOCfPN3DA3jcR1D7IrN0wLjsHo'})
# prozhito_topic_modelling.GetContentFile('prozhito_topic_modelling.pkl')

# Результаты
[Визуализация результатов](https://colab.research.google.com/drive/1rejmaQAdDrRG6KLD1cb4T382OfaVs_Rh?usp=sharing)<br>
[Результаты в виде таймлайна](https://www.sutori.com/story/zhivyie-dushi-xx-vieka--LVLEntAwko13P3cmo1Chy9D8)
## Первая революция
### (*янв. 1905 – янв. 1907*)
Темы дневниковых записей:

In [31]:
pprint(models[0].print_topics())

[(0,
  '0.050*"id" + 0.050*"com" + 0.014*"письмо" + 0.010*"вечер" + 0.007*"день" + '
  '0.006*"ребенок" + 0.006*"читать" + 0.005*"написать" + 0.005*"жена" + '
  '0.005*"писать"'),
 (1,
  '0.015*"это" + 0.013*"говорить" + 0.009*"сказать" + 0.008*"очень" + '
  '0.008*"id" + 0.008*"com" + 0.008*"весь" + 0.007*"который" + 0.007*"»" + '
  '0.006*"свой"'),
 (2,
  '0.010*"весь" + 0.007*"это" + 0.006*"день" + 0.004*"который" + 0.004*"свой" '
  '+ 0.004*"очень" + 0.004*"дом" + 0.004*"сегодня" + 0.004*"наш" + '
  '0.003*"вид"'),
 (3,
  '0.013*"алый" + 0.012*"жена" + 0.009*"м" + 0.009*"б" + 0.007*"п" + '
  '0.006*"самуил" + 0.006*"корпус" + 0.005*"л" + 0.005*"белла" + '
  '0.005*"завтрак"'),
 (4,
  '0.015*"театр" + 0.013*"это" + 0.009*"который" + 0.008*"сегодня" + '
  '0.008*"весь" + 0.007*"2" + 0.007*"com" + 0.007*"id" + 0.006*"говорить" + '
  '0.006*"1"'),
 (5,
  '0.021*"это" + 0.020*"жизнь" + 0.016*"человек" + 0.014*"мочь" + 0.012*"id" + '
  '0.012*"com" + 0.008*"весь" + 0.008*"свой" + 0.008*"

## Период между революциями
### (*февр. 1907 – май 1914*)
Темы дневниковых записей:

In [32]:
pprint(models[1].print_topics())

[(0,
  '0.020*"com" + 0.020*"id" + 0.009*"юрочка" + 0.006*"картина" + 0.005*"Ап" + '
  '0.004*"\xa0" + 0.003*"русский" + 0.003*"искусство" + 0.003*"свой" + '
  '0.003*"музыка"'),
 (1,
  '0.015*"ученица" + 0.013*"класс" + 0.011*"урок" + 0.009*"церковь" + '
  '0.007*"который" + 0.007*"свой" + 0.006*"гимназия" + 0.005*"год" + '
  '0.005*"весь" + 0.005*"ак"'),
 (2,
  '0.015*"вечер" + 0.014*"com" + 0.014*"id" + 0.010*"очень" + '
  '0.010*"приходить" + 0.009*"поехать" + 0.008*"пойти" + 0.008*"милый" + '
  '0.007*"сережа" + 0.007*"приезжать"'),
 (3,
  '0.016*"это" + 0.011*"весь" + 0.009*"который" + 0.008*"говорить" + '
  '0.008*"свой" + 0.008*"очень" + 0.008*"сказать" + 0.006*"мочь" + '
  '0.005*"день" + 0.005*"становиться"'),
 (4,
  '0.020*"com" + 0.020*"id" + 0.020*"это" + 0.019*"жизнь" + 0.018*"человек" + '
  '0.010*"мочь" + 0.009*"свой" + 0.009*"весь" + 0.007*"любовь" + 0.007*"бог"'),
 (5,
  '0.011*"го" + 0.011*"день" + 0.007*"жена" + 0.007*"павловск" + 0.006*"утро" '
  '+ 0.006*"1" + 0.0

## Первая мировая война и две главные революции в России
###(*июнь 1914 – октябрь 1918*)
Темы дневниковых записей:

In [33]:
pprint(models[2].print_topics())

[(0,
  '0.014*"сережа" + 0.013*"очень" + 0.012*"вечер" + 0.012*"приходить" + '
  '0.011*"час" + 0.011*"день" + 0.010*"пойти" + 0.010*"утро" + 0.009*"сегодня" '
  '+ 0.008*"мама"'),
 (1,
  '0.007*"фронт" + 0.006*"дело" + 0.006*"музей" + 0.006*"р" + '
  '0.005*"заседание" + 0.005*"который" + 0.005*"получать" + 0.005*"военный" + '
  '0.005*"армия" + 0.005*"н"'),
 (2,
  '0.011*"весь" + 0.010*"это" + 0.007*"говорить" + 0.006*"день" + 0.005*"свой" '
  '+ 0.005*"»" + 0.005*"человек" + 0.005*"мочь" + 0.005*"становиться" + '
  '0.005*"который"'),
 (3,
  '0.021*"это" + 0.012*"свой" + 0.011*"который" + 0.009*"весь" + 0.009*"мочь" '
  '+ 0.008*"жизнь" + 0.007*"человек" + 0.007*"очень" + 0.005*"время" + '
  '0.004*"знать"'),
 (4,
  '0.102*"id" + 0.102*"com" + 0.013*"ленин" + 0.005*"д" + 0.004*"митинг" + '
  '0.004*"казать" + 0.003*"война" + 0.003*"3" + 0.003*"2" + 0.003*"думка"'),
 (5,
  '0.035*"і" + 0.028*"що" + 0.021*"з" + 0.011*"як" + 0.010*"він" + 0.009*"бо" '
  '+ 0.008*"коли" + 0.007*"це" + 0

## Гражданская война, продразверстка, НЭП
### (*ноябрь 1918 – декабрь 1927*)
Темы дневниковых записей:

In [34]:
pprint(models[3].print_topics())

[(0,
  '0.008*"»" + 0.008*"церковь" + 0.008*"id" + 0.008*"com" + 0.007*"свой" + '
  '0.007*"бекас" + 0.005*"бог" + 0.004*"твой" + 0.004*"божий" + 0.004*"мать"'),
 (1,
  '0.028*"і" + 0.024*"що" + 0.024*"з" + 0.013*"й" + 0.010*"як" + 0.008*"це" + '
  '0.007*"його" + 0.005*"вже" + 0.005*"він" + 0.005*"коли"'),
 (2,
  '0.015*"который" + 0.014*"очень" + 0.008*"концерт" + 0.007*"театр" + '
  '0.007*"играть" + 0.006*"первый" + 0.006*"затем" + 0.005*"пташка" + '
  '0.005*"париж" + 0.005*"вечер"'),
 (3,
  '0.007*"ветер" + 0.007*"день" + 0.006*"весь" + 0.006*"наш" + 0.006*"идти" + '
  '0.006*"дорога" + 0.006*"утро" + 0.005*"ночь" + 0.005*"вода" + '
  '0.005*"большой"'),
 (4,
  '0.015*"день" + 0.015*"вечер" + 0.010*"утро" + 0.009*"сегодня" + 0.009*"час" '
  '+ 0.008*"2" + 0.008*"приходить" + 0.008*"ч" + 0.007*"id" + 0.007*"com"'),
 (5,
  '0.012*"это" + 0.011*"который" + 0.009*"весь" + 0.007*"наш" + 0.007*"свой" + '
  '0.006*"дело" + 0.005*"говорить" + 0.005*"мочь" + 0.004*"россия" + '
  '0.004*"р

## Коллективизация, Большой террор
### (*январь 1928 – август 1939*)
Темы дневниковых записей:

In [35]:
pprint(models[4].print_topics())

[(0,
  '0.018*"м" + 0.018*"д" + 0.018*"2" + 0.015*"1" + 0.015*"ч" + 0.013*"п" + '
  '0.011*"3" + 0.010*"10" + 0.010*"5" + 0.008*"4"'),
 (1,
  '0.125*"com" + 0.125*"id" + 0.007*"the" + 0.007*"of" + 0.006*"консультация" '
  '+ 0.006*"ф" + 0.005*"британский" + 0.004*"оп" + 0.004*"518" + 0.004*"аран"'),
 (2,
  '0.016*"это" + 0.009*"весь" + 0.009*"очень" + 0.009*"говорить" + '
  '0.009*"сказать" + 0.008*"мочь" + 0.006*"знать" + 0.006*"свой" + '
  '0.005*"день" + 0.005*"который"'),
 (3,
  '0.014*"человек" + 0.007*"жизнь" + 0.007*"чемберлен" + 0.006*"церковь" + '
  '0.006*"бог" + 0.006*"русский" + 0.005*"народ" + 0.005*"смерть" + '
  '0.004*"писатель" + 0.004*"свой"'),
 (4,
  '0.022*"это" + 0.015*"свой" + 0.013*"весь" + 0.009*"который" + '
  '0.009*"человек" + 0.008*"мочь" + 0.007*"наш" + 0.005*"время" + 0.005*"дело" '
  '+ 0.005*"самый"'),
 (5,
  '0.035*"день" + 0.019*"сегодня" + 0.015*"час" + 0.014*"вечер" + '
  '0.013*"работа" + 0.013*"весь" + 0.013*"утро" + 0.009*"ходить" + '
  '0.009*"но

## Вторая мировая война
### (*сентябрь 1939 – май 1945*)
Темы дневниковых записей:

In [36]:
pprint(models[5].print_topics())

[(0,
  '0.007*"идти" + 0.006*"і" + 0.006*"немец" + 0.006*"стоять" + 0.006*"дорога" '
  '+ 0.005*"рука" + 0.005*"»" + 0.005*"машина" + 0.005*"наш" + 0.005*"лес"'),
 (1,
  '0.065*"com" + 0.064*"id" + 0.015*"з" + 0.012*"2" + 0.011*"м" + 0.010*"г" + '
  '0.010*"1" + 0.009*"3" + 0.008*"н" + 0.007*"4"'),
 (2,
  '0.009*"й" + 0.005*"він" + 0.005*"Сьогодні" + 0.005*"дружина" + 0.005*"вонь" '
  '+ 0.004*"аля" + 0.004*"the" + 0.004*"було" + 0.004*"був" + 0.004*"від"'),
 (3,
  '0.012*"очень" + 0.009*"письмо" + 0.009*"вечер" + 0.009*"приходить" + '
  '0.008*"сегодня" + 0.008*"говорить" + 0.007*"пойти" + 0.006*"мама" + '
  '0.006*"сказать" + 0.006*"получать"'),
 (4,
  '0.019*"город" + 0.012*"наш" + 0.011*"день" + 0.011*"час" + 0.010*"ночь" + '
  '0.009*"весь" + 0.009*"дом" + 0.007*"снаряд" + 0.007*"самолет" + '
  '0.007*"улица"'),
 (5,
  '0.017*"наш" + 0.009*"немец" + 0.008*"командир" + 0.008*"бой" + 0.008*"весь" '
  '+ 0.007*"полк" + 0.006*"день" + 0.006*"противник" + 0.006*"й" + '
  '0.005*"дивизи

## Промежуточный период истории, заканчивается «оттепелью» 
### (*1946 – 1963*)
Темы дневниковых записей:

In [37]:
pprint(models[6].print_topics())

[(0,
  '0.020*"день" + 0.012*"получать" + 0.011*"работа" + 0.011*"письмо" + '
  '0.009*"весь" + 0.008*"2" + 0.008*"работать" + 0.008*"сегодня" + 0.008*"го" '
  '+ 0.008*"ч"'),
 (1,
  '0.020*"і" + 0.012*"з" + 0.008*"що" + 0.006*"олхо" + 0.005*"хрущов" + '
  '0.005*"це" + 0.004*"м" + 0.004*"як" + 0.004*"питання" + 0.004*"горский"'),
 (2,
  '0.024*"1" + 0.023*"2" + 0.019*"3" + 0.018*"5" + 0.015*"4" + 0.009*"6" + '
  '0.009*"игада" + 0.009*"колхоз" + 0.007*"бригада" + 0.006*"7"'),
 (3,
  '0.022*"это" + 0.012*"весь" + 0.010*"свой" + 0.009*"говорить" + 0.009*"мочь" '
  '+ 0.009*"сказать" + 0.008*"человек" + 0.007*"год" + 0.006*"который" + '
  '0.006*"знать"'),
 (4,
  '0.009*"работа" + 0.008*"который" + 0.007*"год" + 0.007*"вопрос" + 0.006*"г" '
  '+ 0.005*"наш" + 0.005*"космонавт" + 0.005*"полет" + 0.005*"весь" + '
  '0.004*"цк"'),
 (5,
  '0.039*"рука" + 0.029*"вера" + 0.026*"б" + 0.023*"еле" + 0.022*"петровна" + '
  '0.020*"работать" + 0.019*"описной" + 0.019*"Отд" + 0.018*"рукопись" + '
  

## Застой
### (*1964 – май 1987*)
Темы дневниковых записей:

In [38]:
pprint(models[7].print_topics())

[(0,
  '0.016*"фильм" + 0.012*"театр" + 0.010*"играть" + 0.009*"спектакль" + '
  '0.008*"сцена" + 0.008*"концерт" + 0.007*"зал" + 0.007*"актер" + 0.007*"это" '
  '+ 0.007*"очень"'),
 (1,
  '0.016*"com" + 0.016*"id" + 0.013*"год" + 0.010*"г" + 0.008*"книга" + '
  '0.008*"м" + 0.008*"письмо" + 0.007*"написать" + 0.006*"статья" + '
  '0.006*"писать"'),
 (2,
  '0.014*"работа" + 0.010*"г" + 0.009*"вопрос" + 0.008*"план" + 0.008*"ссср" + '
  '0.007*"строительство" + 0.006*"эфрос" + 0.006*"министр" + 0.005*"развитие" '
  '+ 0.005*"афганский"'),
 (3,
  '0.022*"это" + 0.013*"человек" + 0.012*"весь" + 0.010*"который" + '
  '0.010*"свой" + 0.009*"жизнь" + 0.008*"мочь" + 0.005*"мир" + 0.005*"время" + '
  '0.005*"самый"'),
 (4,
  '0.017*"это" + 0.010*"сказать" + 0.010*"весь" + 0.010*"говорить" + '
  '0.009*"свой" + 0.007*"»" + 0.007*"очень" + 0.007*"мочь" + 0.007*"человек" + '
  '0.006*"год"'),
 (5,
  '0.017*"вечер" + 0.013*"день" + 0.010*"утро" + 0.009*"приезжать" + '
  '0.006*"ходить" + 0.006*"ра

## Перестройка и развал СССР
### (*июнь 1987 – июнь 1990*)
Темы дневниковых записей:

In [40]:
pprint(models[8].print_topics())

[(0,
  '0.025*"это" + 0.011*"весь" + 0.009*"говорить" + 0.008*"свой" + '
  '0.007*"сказать" + 0.006*"мочь" + 0.005*"дело" + 0.005*"наш" + '
  '0.005*"человек" + 0.004*"хотеть"'),
 (1,
  '0.010*"это" + 0.010*"который" + 0.008*"человек" + 0.008*"свой" + '
  '0.008*"год" + 0.007*"весь" + 0.006*"жизнь" + 0.005*"время" + 0.005*"жить" + '
  '0.005*"дом"'),
 (2,
  '0.011*"горбачев" + 0.008*"это" + 0.007*"съезд" + 0.007*"совет" + '
  '0.006*"который" + 0.006*"партия" + 0.005*"вопрос" + 0.005*"цк" + '
  '0.005*"свой" + 0.005*"весь"'),
 (3,
  '0.015*"com" + 0.015*"id" + 0.009*"ирка" + 0.005*"москва" + 0.005*"музей" + '
  '0.004*"вечер" + 0.004*"жена" + 0.003*"выставка" + 0.003*"пить" + '
  '0.003*"утро"'),
 (4,
  '0.010*"»" + 0.010*"день" + 0.010*"вечер" + 0.009*"говорить" + 0.009*"очень" '
  '+ 0.009*"приезжать" + 0.009*"сказать" + 0.009*"галя" + 0.007*"час" + '
  '0.007*"весь"'),
 (5,
  '0.006*"река" + 0.004*"госпиталь" + 0.004*"берег" + 0.003*"землетрясение" + '
  '0.003*"маршрут" + 0.003*"м"

## После СССР, «лихие 90-е»
### (*1991 – 2001*)
Темы дневниковых записей:

In [41]:
pprint(models[9].print_topics())

[(0,
  '0.006*"книга" + 0.005*"журнал" + 0.005*"диссертация" + 0.005*"г" + '
  '0.004*"выступать" + 0.004*"работа" + 0.004*"написать" + 0.004*"»" + '
  '0.004*"доклад" + 0.003*"рукопись"'),
 (1,
  '0.007*"зюганов" + 0.007*"булат" + 0.006*"валентина" + 0.005*"концерт" + '
  '0.005*"ельцинский" + 0.004*"р" + 0.004*"запись" + 0.004*"дирижер" + '
  '0.004*"№" + 0.003*"ельцинист"'),
 (2,
  '0.011*"это" + 0.008*"говорить" + 0.007*"м" + 0.007*"день" + 0.006*"»" + '
  '0.006*"вчера" + 0.006*"сказать" + 0.006*"весь" + 0.005*"дом" + '
  '0.004*"сегодня"'),
 (3,
  '0.024*"это" + 0.012*"весь" + 0.011*"свой" + 0.010*"жизнь" + 0.010*"мочь" + '
  '0.010*"человек" + 0.006*"понимать" + 0.005*"бог" + 0.005*"знать" + '
  '0.005*"душа"'),
 (4,
  '0.010*"сказать" + 0.008*"свой" + 0.007*"говорить" + 0.007*"который" + '
  '0.007*"человек" + 0.006*"»" + 0.006*"очень" + 0.006*"год" + 0.006*"это" + '
  '0.005*"день"'),
 (5,
  '0.013*"ельцин" + 0.008*"президент" + 0.007*"россия" + 0.007*"весь" + '
  '0.006*"сове

По результатам заметно, что данные не очень хорошо очищены. :(